# Colab

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/uni_bonn/nlp/final_project

/content/drive/MyDrive/uni_bonn/nlp/final_project


# Install modules

In [ ]:
!pip install datasets
!pip install 'transformers[torch]'
!pip install torchmetrics
!pip install nltk sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.5 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 840.2/840.2 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.3/106.3 kB 3.2 MB/s eta 0:00:00


# Import modules

In [ ]:
import pandas as pd
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import numpy as np
import json
from transformers import EncoderDecoderModel
import torchmetrics
from datasets import Dataset
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
import gc
import torch

In [ ]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [ ]:
import math
import sacrebleu
import nltk
from nltk.translate.bleu_score import sentence_bleu
from nltk import ngrams
from nltk.stem import PorterStemmer
from nltk.corpus import wordnet
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

# Set model

In [ ]:
model_name = "patrickvonplaten/bert2bert-cnn_dailymail-fp16"

# Load dataset using Transformer datasets

In [ ]:
data_files={
    "train":"./dataset/GYAFC_Corpus/data_train.json",
    "val":"./dataset/GYAFC_Corpus/data_val.json",
    "test":"./dataset/GYAFC_Corpus/data_test.json",
}

In [ ]:
dataset = load_dataset("json", data_files=data_files)

Generating train split: 0 examples [00:00, ? examples/s]

Generating val split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/3.22k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

In [ ]:
encoder_max_length=64
decoder_max_length=64

def preprocess_function(examples, input_field="informal", target_field="formal.ref0"):
  inputs = [ex[input_field] for ex in examples["transformation"]]
  targets = [ex[target_field] for ex in examples["transformation"]]

  inputs = tokenizer(inputs, text_target=targets, padding="max_length", truncation=True, max_length=encoder_max_length)
  outputs = tokenizer(targets, padding="max_length", truncation=True, max_length=decoder_max_length)

  examples["input_ids"] = inputs.input_ids
  examples["attention_mask"] = inputs.attention_mask
  examples["decoder_input_ids"] = outputs.input_ids
  examples["decoder_attention_mask"] = outputs.attention_mask
  examples["labels"] = outputs.input_ids.copy()

  # because BERT automatically shifts the labels, the labels correspond exactly to `decoder_input_ids`.
  # We have to make sure that the PAD token is ignored
  # replace padding token id's of the labels by -100 so it's ignored by the loss
  examples["labels"] = [[-100 if token == tokenizer.pad_token_id else token for token in labels] for labels in examples["labels"]]

  return examples

In [ ]:
def create_multi_ref_dataset(dataset):
  for i, target_field in enumerate(['formal.ref0', 'formal.ref1', 'formal.ref2', 'formal.ref3']):
    new_dataset = dataset.map(
      preprocess_function,
      batched=True,
      fn_kwargs=dict(
          target_field=target_field
      )
    )

    if i == 0:
      dataset = dataset.add_column('input_ids', new_dataset['input_ids'])
      # dataset = dataset.add_column('token_type_ids', new_dataset['token_type_ids'])
      dataset = dataset.add_column('attention_mask', new_dataset['attention_mask'])
      dataset = dataset.add_column('decoder_input_ids', new_dataset['decoder_input_ids'])
      dataset = dataset.add_column('decoder_attention_mask', new_dataset['decoder_attention_mask'])
      dataset = dataset.add_column('labels', new_dataset['labels'])
    else:
      dataset = dataset.add_column(f'labels_{i}', new_dataset['labels'])

  return dataset

In [ ]:
val_dataset = create_multi_ref_dataset(dataset['val'])
test_dataset = create_multi_ref_dataset(dataset['test'])

Map:   0%|          | 0/5665 [00:00<?, ? examples/s]

Map:   0%|          | 0/5665 [00:00<?, ? examples/s]

Map:   0%|          | 0/5665 [00:00<?, ? examples/s]

Map:   0%|          | 0/5665 [00:00<?, ? examples/s]

Map:   0%|          | 0/2748 [00:00<?, ? examples/s]

Map:   0%|          | 0/2748 [00:00<?, ? examples/s]

Map:   0%|          | 0/2748 [00:00<?, ? examples/s]

Map:   0%|          | 0/2748 [00:00<?, ? examples/s]

In [ ]:
train_dataset = dataset['train'].map(
    preprocess_function,
    batched=True,
)

Map:   0%|          | 0/104562 [00:00<?, ? examples/s]

In [ ]:
print(val_dataset)
print(train_dataset)

Dataset({
    features: ['transformation', 'id', 'topic', 'input_ids', 'attention_mask', 'decoder_input_ids', 'decoder_attention_mask', 'labels', 'labels_1', 'labels_2', 'labels_3'],
    num_rows: 5665
})
Dataset({
    features: ['transformation', 'id', 'topic', 'input_ids', 'attention_mask', 'decoder_input_ids', 'decoder_attention_mask', 'labels'],
    num_rows: 104562
})


In [ ]:
print(val_dataset[0])

{'transformation': {'formal.ref0': 'If you are under 18, you have a big problem.', 'formal.ref1': 'You have a big problem if you are under eighteen.', 'formal.ref2': 'In the event your age is below 18 years, you are saddled with an enormous difficulty.', 'formal.ref3': 'If you are under the age of 18 you have a big problem.', 'informal': 'if ur under 18 u have a BIG PROBLEM.'}, 'id': 0, 'topic': 'Family_Relationships', 'input_ids': [101, 2065, 24471, 2104, 2324, 1057, 2031, 1037, 2502, 3291, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'decoder_input_ids': [101, 2065, 2017, 2024, 2104, 2324, 1010, 2017, 2031, 1037, 2502, 3291, 1012, 102, 0, 0, 0, 0, 0, 0, 0

In [ ]:
print(len(val_dataset))

5665


In [ ]:
tokenizer.decode(train_dataset[1708]["input_ids"], skip_special_tokens=True)

'besides how old were u when u lost virginity and the tables turn on u now!! ( lol )!'

# Metrics implementations

## Formality

In [ ]:
import pandas as pd
from nltk import ngrams
from collections import defaultdict

In [ ]:
def read_data():
    df_list=[]
    for n in range(1,6):
        df = pd.read_csv(f'./Metrics/Formality/annotated_ngrams/{n}-gram.txt', sep='\t')
        df_list.append(df)
    return df_list

In [ ]:
def evaluate_sentence(sentence, ngram_data_list):
    average_scores=[]
    # Divide the sentence into n-grams
    for n in range(1, 6):
        # Initialize total score and count for averaging
        total_score = 0
        ngrams_list = list(ngrams(sentence.split(), n))
        count = 0
        for ngram in ngrams_list:
            ngram_key = ' '.join(ngram)
            # Check if the n-gram entry exists in the aggregated data
            if ngram_key in ngram_data_list[n-1]['n-gram'].values:
                # Get the total score and total occurrences from the aggregated data
                ngram_entry = ngram_data_list[n-1][ngram_data_list[n-1]['n-gram'] == ngram_key]
                total_score += ngram_entry['average_score'].values[0]
                count += 1
        # Calculate the average score for the sentence
        average_score = total_score / count if count > 0 else 0
        average_scores.append(average_score)
    return average_scores

In [ ]:
# Example usage
sentence = "Mr. Ramesh said 70 percent of India's iron ore lay in states infiltrated by Maoists; production in this area is stalled at 16 million tons a year even though the area has the potential to produce 100 million tons."
formality_evaluation_data = read_data()
result = evaluate_sentence(sentence, formality_evaluation_data)
print(f"Sentence: `{sentence}`\nScore: {result}")

Sentence: `Mr. Ramesh said 70 percent of India's iron ore lay in states infiltrated by Maoists; production in this area is stalled at 16 million tons a year even though the area has the potential to produce 100 million tons.`
Score: [0.8237399883653753, 1.504411302878411, 1.7238738738738726, 1.7999999999999987, 1.7999999999999987]


## Translation Edit Rate Plus (TERp): TERp is calculated by measuring the matching flaw between machine-generated translations and human-created translation. The calculation could be represented as:

In [ ]:
#Higher is better
def calculate_terp(hypothesis: str, reference: str, phrase_table: dict = None, edit_costs: dict = None) -> float:
    """
    Calculate TERp (Translation Edit Rate with partial credit) score between a hypothesis and a reference.

    Parameters:
    - hypothesis (str): The generated hypothesis or translation.
    - reference (str): The reference or ground truth translation.
    - phrase_table (dict, optional): A dictionary representing a phrase table for paraphrase information.
    - edit_costs (dict, optional): A dictionary containing weights for edit operations in the TERp calculation.

    Returns:
    - float: The TERp score, a value indicating the similarity between the hypothesis and the reference,
             where higher values are better.
    """
    hypothesis_tokens = hypothesis.split()
    reference_tokens = reference.split()

    # TERp by Stem Matches, Synonym Matches, and Phrase Substitutions
    stem_matches = calculate_stem_matches(hypothesis_tokens, reference_tokens)
    synonym_matches = calculate_synonym_matches(hypothesis_tokens, reference_tokens)

    phrase_substitutions = calculate_phrase_substitutions(hypothesis_tokens, reference_tokens, phrase_table, edit_costs) if phrase_table is not None and edit_costs is not None else 0


    # Calculate TERp score
    terp_score = (stem_matches + synonym_matches + phrase_substitutions) / (2*len(hypothesis_tokens))

    return terp_score

def calculate_stem_matches(hypothesis_tokens: list, reference_tokens: list) -> int:
    """
    Calculate the number of stem matches between two tokenized sequences.

    Parameters:
    - hypothesis_tokens (list): List of tokens in the hypothesis.
    - reference_tokens (list): List of tokens in the reference.

    Returns:
    - int: The number of stem matches.
    """
    stemmer = PorterStemmer()
    stem_matches = sum(1 for hyp_token, ref_token in zip(hypothesis_tokens, reference_tokens)
                      if stemmer.stem(hyp_token.lower()) == stemmer.stem(ref_token.lower()))
    return stem_matches

def calculate_synonym_matches(hypothesis_tokens: list, reference_tokens: list) -> int:
    """
    Calculate the number of synonym matches between two tokenized sequences.

    Parameters:
    - hypothesis_tokens (list): List of tokens in the hypothesis.
    - reference_tokens (list): List of tokens in the reference.

    Returns:
    - int: The number of synonym matches.
    """
    synonym_matches = sum(1 for hyp_token, ref_token in zip(hypothesis_tokens, reference_tokens)
                          if are_synonyms(hyp_token.lower(), ref_token.lower()))
    return synonym_matches

def are_synonyms(word1: str, word2: str) -> bool:
    """
    Check if two words are synonyms.

    Parameters:
    - word1 (str): The first word.
    - word2 (str): The second word.

    Returns:
    - bool: True if the words are synonyms, False otherwise.
    """
    synsets1 = wordnet.synsets(word1)
    synsets2 = wordnet.synsets(word2)

    return any(set1.wup_similarity(set2) > 0.7 for set1 in synsets1 for set2 in synsets2)

def calculate_phrase_substitutions(hypothesis_tokens: list, reference_tokens: list, phrase_table: dict, edit_costs: dict) -> float:
    """
    Calculate the cost of phrase substitutions between two tokenized sequences.

    Parameters:
    - hypothesis_tokens (list): List of tokens in the hypothesis.
    - reference_tokens (list): List of tokens in the reference.
    - phrase_table (dict): A dictionary representing a phrase table for paraphrase information.
    - edit_costs (dict): A dictionary containing weights for edit operations in the TERp calculation.

    Returns:
    - float: The total cost of phrase substitutions.
    """
    substitution_cost = 0

    for i in range(len(hypothesis_tokens)):
        for j in range(len(reference_tokens)):
            if (hypothesis_tokens[i], reference_tokens[j]) in phrase_table:
                # Retrieve paraphrase information from the phrase table
                paraphrase_info = phrase_table[(hypothesis_tokens[i], reference_tokens[j])]

                # Calculate the cost using the provided formula
                cost = (
                    edit_costs['w1'] +
                    edit_costs['w2'] * paraphrase_info['edit'] * math.log(paraphrase_info['probability']) +
                    edit_costs['w3'] * paraphrase_info['edit'] * paraphrase_info['probability'] +
                    edit_costs['w4'] * paraphrase_info['edit']
                )

                # Ensure the substitution cost is not negative
                substitution_cost += max(0, cost)

    return substitution_cost

def terp_alignment(hypothesis: str, reference: str, phrase_table: dict = None, edit_costs: dict = None) -> list:
    """
    Generate a word-level alignment between a hypothesis and a reference.

    Parameters:
    - hypothesis (str): The generated hypothesis or translation.
    - reference (str): The reference or ground truth translation.
    - phrase_table (dict, optional): A dictionary representing a phrase table for paraphrase information.
    - edit_costs (dict, optional): A dictionary containing weights for edit operations in the TERp calculation.

    Returns:
    - list of tuples: A list of tuples representing the word-level alignment, each tuple contains
                      (hypothesis_token, reference_token, alignment_type).
    """
    alignment = []

    for hyp_token, ref_token in zip(hypothesis.split(), reference.split()):
        if hyp_token == ref_token:
            alignment.append((hyp_token, ref_token, "Exact Match"))
        else:
            alignment.append((hyp_token, ref_token, "Mismatch"))

    return alignment

## Paraphrase In N-gram Changes (PINC): PINC is calculated by computing the percentage of n-grams that appear in the candidate sentence but not in the source sentence. The calculation could be represented as:

In [ ]:
# Lower is better
def calculate_pinc(hypothesis: str, reference: str, n: int) -> float:
    """
    Calculate PINC (Precision-based n-gram Inclusion Count) score between a hypothesis and a reference.

    Parameters:
    - hypothesis (str): The generated hypothesis or translation.
    - reference (str): The reference or ground truth translation.
    - n (int): The size of n-grams for which to calculate the PINC score.

    Returns:
    - float: The PINC score, where lower values indicate better similarity to the reference.
    """
    hypothesis_split = hypothesis.split()
    reference_split = reference.split()

    hypothesis_ngrams = set(ngrams(hypothesis_split, n))
    reference_ngrams = set(ngrams(reference_split, n))
    new_ngrams = hypothesis_ngrams - reference_ngrams
    hypo_ngram_len = len(hypothesis_ngrams)
    pinc_score = len(new_ngrams) / hypo_ngram_len if hypo_ngram_len != 0 else 0

    return pinc_score

## Test

In [ ]:
# Example usage:
hypothesis_sentence = "This is an example sentence."
reference_sentence = "This is an example sentence."

# Calculate TERp score
terp_score = calculate_terp(hypothesis_sentence, reference_sentence)
print(f"TERp Score: {terp_score}")

# Generate alignment
alignment = terp_alignment(hypothesis_sentence, reference_sentence)
print("Alignment:", alignment)

TERp Score: 0.8
Alignment: [('This', 'This', 'Exact Match'), ('is', 'is', 'Exact Match'), ('an', 'an', 'Exact Match'), ('example', 'example', 'Exact Match'), ('sentence.', 'sentence.', 'Exact Match')]


In [ ]:
pinc_score = calculate_pinc(hypothesis_sentence, reference_sentence, 2)
print(f"PINC Score: {pinc_score}")

PINC Score: 0.0


## Aggregate functions

In [ ]:
def terp_metric(preds, refs):
  scores = np.zeros(len(preds), dtype=float)
  for i in range(len(preds)):
    pred = preds[i]
    ref = refs[i]
    score = np.min(np.array(list(map(lambda x: calculate_terp(pred, x), ref))))
    scores[i] = score

  return np.mean(scores)

In [ ]:
def pinc_metric(preds, refs, n=2):
  scores = np.zeros(len(preds), dtype=float)
  for i in range(len(preds)):
    pred = preds[i]
    ref = refs[i]
    score = np.min(np.array(list(map(lambda x: calculate_pinc(pred, x, n), ref))))
    scores[i] = score

  return np.mean(scores)

In [ ]:
def formality_metric(preds):
  formality_evaluation_data = read_data()
  formality_scores = np.array(list(map(lambda x: evaluate_sentence(x, formality_evaluation_data), preds)))
  return np.mean(formality_scores)

# Compute total metric

In [ ]:
def compute_metrics(pred_str, label_str):
  preds = [" ".join([token.text for token in nlp(line)]) for line in pred_str]
  labels = [[" ".join([token.text for token in nlp(line)]) for line in references] for references in label_str]

  # Calculate metrics
  bleu1 = torchmetrics.text.BLEUScore(n_gram=1)
  bleu2 = torchmetrics.text.BLEUScore(n_gram=2)
  bleu3 = torchmetrics.text.BLEUScore(n_gram=3)
  bleu4 = torchmetrics.text.BLEUScore(n_gram=4)
  sacrebleu = torchmetrics.text.SacreBLEUScore()
  ter = torchmetrics.text.TranslationEditRate(normalize=True, lowercase=False)

  bleu1(preds, labels)
  bleu2(preds, labels)
  bleu3(preds, labels)
  bleu4(preds, labels)
  ter(preds, labels)
  sacrebleu(preds, labels)

  return dict(
      bleu1=bleu1.compute(),
      bleu2=bleu2.compute(),
      bleu3=bleu3.compute(),
      bleu4=bleu4.compute(),
      ter=ter.compute(),
      sacrebleu=sacrebleu.compute(),
      terp=terp_metric(preds, labels),
      pinc=pinc_metric(preds, labels, n=4),
      formality=formality_metric(preds)
  )

# Inference


In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained("./model_output/bert2bert/raw/checkpoint-10", device_map="cuda")

Some weights of EncoderDecoderModel were not initialized from the model checkpoint at ./model_output/bert2bert/raw/checkpoint-10 and are newly initialized: ['decoder.cls.predictions.decoder.weight', 'decoder.cls.predictions.decoder.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
print(model.config)

EncoderDecoderConfig {
  "_name_or_path": "./model_output/bert2bert/raw/checkpoint-10",
  "architectures": [
    "EncoderDecoderModel"
  ],
  "decoder": {
    "_name_or_path": "",
    "add_cross_attention": true,
    "architectures": [
      "BertForMaskedLM"
    ],
    "attention_probs_dropout_prob": 0.1,
    "bad_words_ids": null,
    "begin_suppress_tokens": null,
    "bos_token_id": null,
    "chunk_size_feed_forward": 0,
    "classifier_dropout": null,
    "cross_attention_hidden_size": null,
    "decoder_start_token_id": null,
    "diversity_penalty": 0.0,
    "do_sample": false,
    "early_stopping": false,
    "encoder_no_repeat_ngram_size": 0,
    "eos_token_id": null,
    "exponential_decay_length_penalty": null,
    "finetuning_task": null,
    "forced_bos_token_id": null,
    "forced_eos_token_id": null,
    "gradient_checkpointing": false,
    "hidden_act": "gelu",
    "hidden_dropout_prob": 0.1,
    "hidden_size": 768,
    "id2label": {
      "0": "LABEL_0",
      "1": "L

In [ ]:
train_dataset.set_format(
    type="torch", columns=["input_ids", "attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels"],
)
val_dataset.set_format(
    type="torch", columns=["input_ids", "attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels", 'labels_1', 'labels_2', 'labels_3'],
)
test_dataset.set_format(
    type="torch", columns=["input_ids", "attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels", 'labels_1', 'labels_2', 'labels_3'],
)

In [ ]:
from tqdm import tqdm

In [ ]:
def create_inference(dataset):
  out = list()
  for i in tqdm(range(len(dataset))):
    input_ids = dataset[i:i+1]['input_ids'].to("cuda")
    outputs = model.generate(input_ids, max_length=64)
    edited_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    out.append(edited_text)

  return out

In [ ]:
val_inference = create_inference(val_dataset)

100%|██████████| 5665/5665 [1:44:40<00:00,  1.11s/it]


In [ ]:
import pickle
PIK = "./output/bert2bert_finetuned_val_inference.pkl"
with open(PIK, "wb") as f:
    pickle.dump(val_inference, f)

In [ ]:
test_inference = create_inference(test_dataset)

100%|██████████| 2748/2748 [50:55<00:00,  1.11s/it]


In [ ]:
import pickle

In [ ]:
PIK = "./output/bert2bert_finetuned_test_inference.pkl"
with open(PIK, "wb") as f:
    pickle.dump(test_inference, f)

# Validation

In [ ]:
import pickle

PIK = "./output/bert2bert_finetuned_val_inference.pkl"
with open(PIK, "rb") as file:
    val_inference = pickle.load(file)

In [ ]:
PIK = "./output/bert2bert_finetuned_test_inference.pkl"
with open(PIK, "rb") as file:
    test_inference = pickle.load(file)

In [ ]:
def create_reference(dataset):
  data = dataset['transformation']

  ref0 = [data[i]['formal.ref0'] for i in range(len(data))]
  ref1 = [data[i]['formal.ref1'] for i in range(len(data))]
  ref2 = [data[i]['formal.ref2'] for i in range(len(data))]
  ref3 = [data[i]['formal.ref3'] for i in range(len(data))]

  return list(zip(ref0, ref1, ref2, ref3))

In [ ]:
val_references =  create_reference(val_dataset)
test_references =  create_reference(test_dataset)

In [ ]:
print(test_references[1])

('Good luck in your search for your perfect match.', 'Good luck in your search for a new love.', 'Good luck in your search for the one!', 'I wish you luck in your search for "the one".')


In [ ]:
print(test_inference[1])

[unused6] [unused6] [unused6] [unused6] [unused6] [unused6] [unused6] [unused6] [unused6] [unused6] [unused6] [unused6] [unused6] [unused6] [unused6] [unused6] [unused6] [unused6] [unused6] [unused6] [unused6] [unused6] [unused6] [unused6] [unused6] [unused6] [unused6] [unused6] [unused6] [unused6] [unused6] [unused6] [unused6] [unused6] [unused6] [unused6] [unused6] [unused6] [unused6] [unused6] [unused6] [unused6] [unused6] [unused6] [unused6] [unused6] [unused6] [unused6] [unused6] [unused6] [unused6] [unused6] [unused6] [unused6] [unused6] [unused6] [unused6] [unused6] [unused6] [unused6] [unused6] [unused6] [unused4]


## Val set

In [ ]:
val_metrics = compute_metrics(val_inference, val_references)

In [ ]:
print(val_metrics)

{'bleu1': tensor(0.), 'bleu2': tensor(0.), 'bleu3': tensor(0.), 'bleu4': tensor(0.), 'ter': tensor(15.1907), 'sacrebleu': tensor(0.), 'terp': 0.0, 'pinc': 1.0, 'formality': -0.6783723196881056}


In [55]:
PIK = "./output/bert2bert_finetuned_val_metrics.pkl"
with open(PIK, "wb") as f:
    pickle.dump(val_metrics, f)

## Test set

In [ ]:
test_metrics = compute_metrics(test_inference, test_references)

In [54]:
print(test_metrics)

{'bleu1': tensor(0.), 'bleu2': tensor(0.), 'bleu3': tensor(0.), 'bleu4': tensor(0.), 'ter': tensor(14.6883), 'sacrebleu': tensor(0.), 'terp': 0.0, 'pinc': 1.0, 'formality': -0.6783723196881056}


In [56]:
PIK = "./output/bert2bert_finetuned_test_metrics.pkl"
with open(PIK, "wb") as f:
    pickle.dump(test_metrics, f)